## LLM链

In [1]:
! export LANGCHAIN_TRACING_V2="true"
! export LANGCHAIN_API_KEY="..."

### 语言模型

In [2]:
from langchain_community.llms import Ollama
llm = Ollama(model="qwen:4b")
llm.invoke("LangSmith是什么")

'“LangSmith”是一个网络用语，“娘炮”的拼音缩写。'

### 聊天模型的提示模板

用于为聊天模型创建灵活的模板提示。

[langchain_core.prompts.chat.ChatPromptTemplate](langchain_core.prompts.chat.ChatPromptTemplate.ipynb)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位世界一流的技术文档作家。"),
    ("user", "{input}")
])
chain = prompt | llm 
chain.invoke({"input": "Langsmith如何帮助测试？"})

'Langsmith是一种人工智能测试工具，它可以帮助开发者进行软件测试。具体来说，Langsmith可以帮助开发者进行以下几种类型的测试：\n\n1. 功能性测试：Langsmith可以模拟用户操作场景，通过自动化方式执行功能测试。\n\n2. 性能测试：Langsmith可以根据预先设定的性能指标来自动执行性能测试。\n\n3. 安全性测试：Langsmith可以根据预先设定的安全性指标来自动执行安全性测试。\n\n4. 兼容性测试：Langsmith可以根据预先设定的兼容性指标来自动执行兼容性测试。\n\n5. 软件质量测试：Langsmith可以根据预先设定的质量测试指标来自动执行软件质量测试。'

### 输出解析器

用于解析LLM结果到最可能字符串

[langchain_core.output_parsers.string.StrOutputParser](langchain_core.output_parsers.string.StrOutputParser.ipynb)

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"input": "Langsmith如何帮助测试？"})

'Langsmith是一种人工智能技术，它可以用来模拟人类的对话行为。在软件测试中，Langsmith可以帮助测试人员进行更加自然、流畅的交流。这样可以帮助测试人员更快地理解产品的功能和使用方法，从而提高测试效率和质量。'

### 小结

至此，讨论了基本的LLM链，涉及语言模型、提示、输出解析器，更深入的部分，参考

[模型I/O](模型IO.ipynb)

## 检索链



In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [7]:
print(docs[0])

page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand h

In [8]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="shaw/dmeta-embedding-zh:latest",
)

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [11]:
# 直接从文件传入上下文

from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can let you visualize test results. This means that instead of just looking at numbers, you can actually see which parts of your code are giving problems, and where the problems might be coming from. This can be a very useful way to understand and fix problems with your code.'

In [12]:
# 通过向量库检索传入上下文

from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith is an AI platform that helps developers build applications, debug issues, and automate various tasks related to software development.
In the context of testing, LangSmith can be used to perform a variety of testing tasks, such as:

  * Running tests against different versions of your application.
  * Automatically scoring traces, sending them to annotation queues, or sending them to datasets.
  * Automatically identifying important data points in trace view and flagging them for manual review.

In summary, LangSmith is an AI platform that helps developers build applications, debug issues, and automate various tasks related to software development. In the context of testing, LangSmith can be used to perform a variety of testing tasks, such as running tests against different versions of your application automatically scoring traces, sending them to annotation queues, or sending them
